Representation learning (embedding with CNN).

In [1]:
import collections
import os
from collections import Counter
import numpy as np
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import string
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

from argparse import Namespace

In [2]:
args = Namespace(
    train_dataset_csv="train.csv",
    test_dataset_csv="test.csv",
    train_proportion=0.7,
    val_proportion=0.3,
    train_split_csv="train_with_splits.csv", 
    #train_split_csv="train_tweet.csv", 
    glove_patch='C:/Users/Admin/Desktop/NLP/embed/glove.6B.100d.txt',
    seed=123)

            

In [3]:
train_splits=pd.read_csv(args.train_split_csv)
train_splits

,id,keyword,location,text,target,split
0,9815,trauma,LOCAL ATLANTA NEWS 4/28/00 - 4/28/15 FREELANCER,@raabchar_9 @drphil @morganlawgrp how do you s...,0,train
1,4527,emergency,"Anchorage, AK",anchorage jobs emergency medicine - nurse prac...,0,train
2,6540,injury,NaN,joboozoso : usat usatoday_nfl michael floyd '...,0,train
3,1456,body%20bagging,NaN,wwe 9k9 mycareer ep9 tyrone body bagging dudes...,0,train
4,10566,windstorm,she/her/your majesty/empress,i like the weird ones like rain of mystical or...,0,train
...,...,...,...,...,...,...
7478,4742,evacuate,NaN,sooo police dispatch said there was a person t...,1,val
7479,9206,suicide%20bombing,NaN,suicide bombing is just the fear of dying alone,1,val
7480,2540,collision,"North Highlands, CA",traffic collision - ambulance enroute : elkho...,1,val
7481,4747,evacuate,Nashville,@ahhtheenikki and from what i can tell- they r...,1,val


In [4]:
train_splits['keyword']=train_splits.keyword.fillna('_nan_')

Baseline_1

In [5]:
train=train_splits[train_splits.split=='train']
val=train_splits[train_splits.split=='val']
print(len(train), len(val))

5251 2232


model_1. Full representation of the text of the tweet.

The Vocabulary.

In [6]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
            
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index
        
        Args: 
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
   
    def __len__(self):
        return len(self._token_to_idx)

For text: Adding tokens of the beginning, end, unknown word, and the phrase complement symbol to the maximum length (mask).

In [7]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            'unk_index' needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

The Vectorizer.

In [8]:
class TweetVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""    
    def __init__(self, text_vocab, keyword_vocab):
        self.text_vocab = text_vocab
        self.keyword_vocab = keyword_vocab
        
    def vectorize_text(self, text, vector_length=-1):
        """
        Args:
            text (str): the string of words separated by a space
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vetorized text (numpy.array)
        """
        indices = [self.text_vocab.begin_seq_index]
        for token in text.split(" "):
            if (token not in string.punctuation):
                indices.append(self.text_vocab.lookup_token(token))
                
        indices.append(self.text_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.text_vocab.mask_index

        return out_vector
     
    @classmethod
    def from_dataframe(cls, tweet_df, cutoff=3):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            tweet_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary 
        Returns:
            an instance of the TweetVectorizer
        """
        keyword_vocab = Vocabulary()        
        for keyword in list(set(tweet_df.keyword)):
            keyword_vocab.add_token(keyword)

        word_counts = Counter()
        for text in tweet_df.text:
            for token in text.split(" "):
                if (token not in string.punctuation):
                    word_counts[token] += 1
        
        text_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                text_vocab.add_token(word)
        
        return cls(text_vocab, keyword_vocab)

    @classmethod
    def from_serializable(cls, contents):
        text_vocab = \
            SequenceVocabulary.from_serializable(contents['text_vocab'])
        keyword_vocab =  \
            Vocabulary.from_serializable(contents['keyword_vocab'])

        return cls(text_vocab=text_vocab, keyword_vocab=keyword_vocab)

    def to_serializable(self):
        return {'text_vocab': self.text_vocab.to_serializable(),
                'keyword_vocab': self.keyword_vocab.to_serializable()}

The Dataset.

In [9]:
class TweetDataset(Dataset):
    def __init__(self, tweet_df, vectorizer):
        """
        Args:
            tweet_df (pandas.DataFrame): the dataset
            vectorizer (TweetVectorizer): vectorizer instatiated from dataset
        """
        self.tweet_df = tweet_df
        self._vectorizer = vectorizer

        # +1 if only using begin_seq, +2 if using both begin and end seq tokens
        def measure_len(text):
            len_=0
            for word in text:
                if word not in string.punctuation:
                    len_+=1
            return len_
                    
        #measure_len = lambda context: len(context.split(" "))
        self._max_seq_length = max(map(measure_len, tweet_df.text)) + 2
        

        self.train_df = self.tweet_df[self.tweet_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.tweet_df[self.tweet_df.split=='val']
        self.validation_size = len(self.val_df)

        
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size)}

        self.set_split('train')

        # Class weights
        class_counts = tweet_df.target.value_counts().to_dict()
        freq = [count for _, count in class_counts.items()]
        self.class_weights = 1.0 / torch.tensor(freq, dtype=torch.float32)
        
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, tweet_csv):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            tweet_csv (str): location of the dataset
        Returns:
            an instance of TweetDataset
        """
        tweet_df = pd.read_csv(tweet_csv)
        train_tweet_df = tweet_df[tweet_df.split=='train']
        return cls(tweet_df, TweetVectorizer.from_dataframe(train_tweet_df))

    
    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        text_vector = \
            self._vectorizer.vectorize_text(row.text, self._max_seq_length)

        keyword_index_from_text = \
            self._vectorizer.text_vocab.lookup_token(row.keyword)
        
        keyword_index_from_keyword= \
            self._vectorizer.keyword_vocab.lookup_token(row.keyword)

        return {'x_data': text_vector,
                'keyword_index_from_text': keyword_index_from_text,
                'keyword_index_from_keyword': keyword_index_from_keyword,
                'y_target': row.target}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size


In [10]:
data=TweetDataset.load_dataset_and_make_vectorizer(args.train_split_csv)

In [11]:
data.__getitem__(34)

{'x_data': array([  2, 278,   6, 279,   1,  14, 115,   6, 147, 280, 281, 139, 282,
        115,   6, 147, 280, 281, 283,   1,   1,   3,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0

In [483]:
train_splits.iloc[236].keyword, train_splits.iloc[236].text

('body%20bags',
 " ' your body will heal the bags under your eyes will go away you\x89ûªll be so happy you\x89ûªll smile and really .  .  .  '  http :  /  / t . co / wukcalnqms")

In [484]:
train_splits[train_splits.keyword=='body%20bags'].head(5)

,id,keyword,location,text,target,split
55,1484,body%20bags,United Kingdom,womens buckle casual stylish shoulder handbags...,0,train
68,1526,body%20bags,Speaking the Truth in Love,fairfax investigating firefighter over faceboo...,0,train
236,1519,body%20bags,NaN,' your body will heal the bags under your eye...,0,train
348,1520,body%20bags,"California, USA",womens handbags cross body geometric pattern s...,0,train
586,1486,body%20bags,CA,bestseller ! fossil dawson mini cross body ba...,0,train


In [485]:
data.__getitem__(236)

{'x_data': array([   2,    9,   38,   78, 1073,   44,  411, 1074,    9, 1075,   78,
         424,  692,    1,   79,  402,  660,    1, 1076,   11, 1077,   24,
          25,   26,    1,    3,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,   

As we can see, the keyword often does not reflect the context of the message at all.

In [12]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    """A generator function which wraps the PyTorch DataLoader. 
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [13]:
gen=generate_batches(data, 9)
t=next(gen)
t

{'x_data': tensor([[   2,   61,  583,  ...,    0,    0,    0],
         [   2,   42,  460,  ...,    0,    0,    0],
         [   2,   42,  638,  ...,    0,    0,    0],
         ...,
         [   2,    8, 1443,  ...,    0,    0,    0],
         [   2, 2288, 1242,  ...,    0,    0,    0],
         [   2,  213,  434,  ...,    0,    0,    0]]),
 'keyword_index_from_text': tensor([ 981,  864, 1305,   31,  818,  475,  843, 2008, 2473]),
 'keyword_index_from_keyword': tensor([148,  92,  82, 100,  59,  95, 201, 182,  88]),
 'y_target': tensor([0, 0, 0, 0, 0, 0, 0, 1, 1])}

In [14]:
t['x_data'].shape

torch.Size([9, 211])

The Model: TweetClassifier.

In [15]:
class TweetClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_channels, 
                 hidden_dim, num_classes, dropout_p, 
                 pretrained_embeddings=None, padding_idx=0):
        """
        Args:
            embedding_size (int): size of the embedding vectors (the size of each embedding vector)
            num_embeddings (int): number of embedding vectors(size of the dictionary of embeddings).
            filter_width (int): width of the convolutional kernels
            num_channels (int): number of convolutional kernels per layer
            hidden_dim (int): the size of the hidden dimension
            num_classes (int): the number of classes in classification
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings
                default is None. If provided, 
            padding_idx (int): an index representing a null position
        """
        super(TweetClassifier, self).__init__()

        if pretrained_embeddings is None:

            self.emb = nn.Embedding(num_embeddings=num_embeddings,
                                    embedding_dim=embedding_size,                                    
                                    padding_idx=padding_idx)        
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.emb = nn.Embedding(num_embeddings=num_embeddings,
                                    embedding_dim=embedding_size,                                    
                                    padding_idx=padding_idx,
                                    _weight=pretrained_embeddings)
        
            
        self.convnet = nn.Sequential(
            nn.Conv1d(in_channels=embedding_size, 
                   out_channels=num_channels, kernel_size=3),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=3, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=3, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=3),
            nn.ELU()
        )

        self._dropout_p = dropout_p
        self.fc1 = nn.Linear(num_channels, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, dataset._max_seq_length)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, num_classes)
        """
        
        # embed and permute so features are channels
        x_embedded = self.emb(x_in).permute(0, 2, 1)

        features = self.convnet(x_embedded)

        # average and remove the extra dimension
        remaining_size = features.size(dim=2)
        features = F.avg_pool1d(features, remaining_size).squeeze(dim=2)        
        features = F.dropout(features, p=self._dropout_p)
        
        # mlp classifier
        intermediate_vector = F.relu(F.dropout(self.fc1(features), p=self._dropout_p))
        prediction_vector = self.fc2(intermediate_vector).squeeze()

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector

In [16]:
import io

In [17]:
def load_glove_from_file(glove_filepath):
    """
    Load the GloVe embeddings 
    
    Args:
        glove_filepath (str): path to the glove embeddings file 
    Returns:
        word_to_index (dict), embeddings (numpy.ndarary)
    """

    word_to_index = {}
    embeddings = []
    #with open(glove_filepath, "r") as fp:
    with io.open(glove_filepath, encoding='utf-8') as fp:
        for index, line in enumerate(fp):
            line = line.split(" ") # each line: word num1 num2 ...
            word_to_index[line[0]] = index # word = line[0] 
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    return word_to_index, np.stack(embeddings)

def make_embedding_matrix(glove_filepath, words):
    """
    Create embedding matrix for a specific set of words.
    
    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataset
    """
    word_to_idx, glove_embeddings = load_glove_from_file(glove_filepath)
    embedding_size = glove_embeddings.shape[1]
    
    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = glove_embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

In [26]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_score': [],
            'val_loss': [],
            'val_score': [],
            'test_loss': -1,
            'test_score': -1,
            'model_filename': args.model_state_file}


Settings and some prep work.

In [18]:
args = Namespace(
    train_dataset_csv="train.csv",
    test_dataset_csv="test.csv",
    train_proportion=0.7,
    val_proportion=0.3,
    train_split_csv="train_with_splits.csv", 
    #train_split_csv="train_tweet.csv",
    glove_patch='C:/Users/Admin/Desktop/NLP/embed/glove.6B.100d.txt',
    model_state_file="model.pth",
    seed=123,
    use_glove=False,
    embedding_size=100, 
    hidden_dim=100, 
    num_channels=100, 
    # Training hyper parameter    
    learning_rate=0.001, 
    dropout_p=0.4, 
    batch_size=64, 
    num_epochs=25, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True
   
) 



In [19]:
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False
    
args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

Using CUDA: True


Initializations.

In [20]:
args.use_glove = True

In [21]:
# create dataset and vectorizer
dataset = TweetDataset.load_dataset_and_make_vectorizer(args.train_split_csv)

vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.text_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(glove_filepath=args.glove_patch, 
                                       words=words)
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None

Using pre-trained embeddings


In [22]:
embeddings.shape

(3404, 100)

In [23]:
classifier = TweetClassifier(embedding_size=args.embedding_size, 
                            num_embeddings=len(vectorizer.text_vocab),
                            num_channels=args.num_channels,
                            hidden_dim=args.hidden_dim, 
                            num_classes=1, 
                            dropout_p=args.dropout_p,
                            pretrained_embeddings=embeddings,
                            #pretrained_embeddings=None,
                            padding_idx=0)
classifier

TweetClassifier(
  (emb): Embedding(3404, 100, padding_idx=0)
  (convnet): Sequential(
    (0): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (3): ELU(alpha=1.0)
    (4): Conv1d(100, 100, kernel_size=(3,), stride=(2,))
    (5): ELU(alpha=1.0)
    (6): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (7): ELU(alpha=1.0)
  )
  (fc1): Linear(in_features=100, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)

In [24]:
def compute_score(y_target, y_pred):
    y_target_nump = y_target.cpu().bool().numpy()
    y_pred_nump = (torch.sigmoid(y_pred)>0.5).cpu().numpy()
    #y_pred_nump = (y_pred>0.5).cpu().long().numpy()
    if y_target_nump.sum()==0:
        zv=1
    else:
        zv=0   
    prec=precision(y_target_nump, y_pred_nump, zero_division=zv)
    rec=recall(y_target_nump, y_pred_nump, zero_division=zv) 
    div=(prec+rec)
    if div!=0:
        return 2*prec*rec/div
    else:
        return 0

Training loop.

In [27]:
classifier = classifier.to(args.device)

loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine', 
                        total=args.num_epochs,
                        position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
                    total=dataset.get_num_batches(args.batch_size), 
                    position=1, 
                    leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_score = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].long())

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the score
            score_t = compute_score(batch_dict['y_target'], y_pred)
            running_score += (score_t - running_score) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, 
                                  score=running_score, 
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_score'].append(running_score)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_score = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].long())

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the score
            #print(prec_(batch_dict['y_target'], y_pred))
            score_t = compute_score(batch_dict['y_target'], y_pred)
            running_score += (score_t - running_score) / (batch_index + 1)
            
            val_bar.set_postfix(loss=running_loss, 
                                score=running_score, 
                                epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_score'].append(running_score)
       
        scheduler.step(train_state['val_loss'][-1])

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
        
except KeyboardInterrupt:
    print("Exiting loop")


training routine:   0%|          | 0/25 [00:00<?, ?it/s]

split=train:   0%|          | 0/82 [00:00<?, ?it/s]

split=val:   0%|          | 0/34 [00:00<?, ?it/s]

In [28]:
train_state['val_score']

[0.0,
 0.7385222457884018,
 0.7535773875044153,
 0.759608385539358,
 0.756930274577399,
 0.7576011972740779,
 0.7548592365510065,
 0.7428751779014835,
 0.7426613218666673,
 0.7469590930691756,
 0.7435974203559805,
 0.7421036918538861,
 0.7461625954089586,
 0.7449694847831401,
 0.7448504503001893,
 0.7455513202368156,
 0.7530394989300809,
 0.742712813691442,
 0.7441155559014379,
 0.743995868403947,
 0.7588657078908256,
 0.7482498152665928,
 0.7391288409193517,
 0.7482625745337821,
 0.7407728080956453]